<a href="https://colab.research.google.com/github/WasudeoGurjalwar/AGENTIC_AI_TRAININGS_Rocky/blob/main/EV_Charging_Station_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Implement a multi-agent customer support workflow using LangGraph. The workflow should include a Classifier Agent to route queries (billing, technical, general) to specialized agents (Billing Agent, Technical Agent, General Agent). The final responses from the specialized agents should be aggregated into a single answer. Use Google Gemini (gemini-2.5-flash-lite) and OpenAI (gpt-4o-mini) models, initializing them in separate blocks after installing necessary libraries and configuring API keys.

## Install necessary libraries

### Subtask:
Install LangChain, LangGraph, and the libraries for interacting with Google Gemini and OpenAI APIs.


**Reasoning**:
Install the necessary libraries using pip.



In [ ]:
%pip install --quiet langchain langgraph langchain-google-genai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


## Configure api keys

### Subtask:
Add code to securely load and configure API keys for Google Gemini and OpenAI.


**Reasoning**:
The subtask is to securely load and configure API keys for Google Gemini and OpenAI. This involves importing `getpass`, prompting the user for the keys, and setting them as environment variables. These steps can be done in a single code block.



In [ ]:
import getpass
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Initialize language models

### Subtask:
Initialize two LLMs: one from Google (Gemini) and one from OpenAI (GPT).


**Reasoning**:
Import the necessary classes and instantiate the Google Gemini and OpenAI GPT models as specified in the instructions.



In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")
openai_llm = ChatOpenAI(model="gpt-5-mini")

## Define agent tools (optional but recommended)

### Subtask:
Although not explicitly requested, defining tools for each agent (e.g., knowledge base lookups, external API calls) will enhance their capabilities. This step will be marked as optional in the plan.


**Reasoning**:
Define placeholder functions for the tools that each agent might use.



In [ ]:
def knowledge_base_lookup_tool(query: str) -> str:
  """Tool for searching a knowledge base for technical issues."""
  print(f"Searching knowledge base for: {query}")
  # Placeholder implementation
  return "Information from knowledge base related to " + query

def billing_information_tool(customer_id: str) -> str:
  """Tool for accessing billing information."""
  print(f"Accessing billing information for customer ID: {customer_id}")
  # Placeholder implementation
  return "Billing details for customer ID " + customer_id

def general_information_tool(query: str) -> str:
  """Tool for retrieving general information."""
  print(f"Retrieving general information for: {query}")
  # Placeholder implementation
  return "General information about " + query

# The classifier agent might not need a specific tool beyond the LLM itself for routing.
# However, if it needed to look up customer history to help with classification,
# a tool could be defined here.

## Define agent nodes

### Subtask:
Create a node for each agent (Classifier, Billing, Technical, General). Each node will contain the logic for that agent.


**Reasoning**:
Define the Python functions for each agent (Classifier, Billing, Technical, General) as described in the instructions, using the previously initialized language models.



In [ ]:
from langgraph.graph import StateGraph
from typing import TypedDict, Annotated, Union
import operator
from langchain_core.prompts import ChatPromptTemplate

# Define a state for the graph using TypedDict
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        query: initial customer query
        classification: classification of the query
        billing_response: str
        technical_response: str
        general_response: str
        final_response: str
    """
    query: str
    classification: str
    billing_response: str
    technical_response: str
    general_response: str
    final_response: str

# Redefine the classifier_agent to return a dictionary to update the state
def classifier_agent(state: dict) -> dict:
    """Classifies the customer query and updates the state."""
    query = state['query']
    print(f"--- Classifier Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that classifies customer queries into one of the following categories: billing, technical, general. Respond with only the category name."),
        ("human", "Classify the following query: {query}")
    ])
    chain = prompt | gemini_llm
    category = chain.invoke({"query": query}).content.strip().lower() # Ensure clean output
    # Basic parsing to extract the category
    if "billing" in category:
        classified_category = "billing"
    elif "technical" in category:
        classified_category = "technical"
    else:
        classified_category = "general"

    print(f"Classified as: {classified_category}")
    print(f"Output State Update: {{'classification': '{classified_category}'}}")
    print(f"--- Classifier Agent End ---")

    # Return a dictionary to update the state
    return {"classification": classified_category}

# Redefine the specialized agents to return dictionaries to update the state
def billing_agent(state: dict) -> dict:
    """Handles billing queries and updates the state."""
    query = state['query']
    print(f"--- Billing Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that responds to billing queries."),
        ("human", "Respond to the following billing query: {query}")
    ])
    chain = prompt | openai_llm
    response = chain.invoke({"query": query}).content
    print(f"Response: {response}")
    print(f"Output State Update: {{'billing_response': '{response}'}}")
    print(f"--- Billing Agent End ---")
    return {"billing_response": response}

def technical_agent(state: dict) -> dict:
    """Handles technical queries and updates the state."""
    query = state['query']
    print(f"--- Technical Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that responds to technical queries."),
        ("human", "Respond to the following technical query: {query}")
    ])
    chain = prompt | openai_llm
    response = chain.invoke({"query": query}).content
    print(f"Response: {response}")
    print(f"Output State Update: {{'technical_response': '{response}'}}")
    print(f"--- Technical Agent End ---")
    return {"technical_response": response}

def general_agent(state: dict) -> dict:
    """Handles general queries and updates the state."""
    query = state['query']
    print(f"--- General Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that responds to general queries."),
        ("human", "Respond to the following general query: {query}")
    ])
    chain = prompt | openai_llm
    response = chain.invoke({"query": query}).content
    print(f"Response: {response}")
    print(f"Output State Update: {{'general_response': '{response}'}}")
    print(f"--- General Agent End ---")
    return {"general_response": response}

## Define router node

### Subtask:
Create a router node that takes the query and the output of the Classifier Agent to determine which Specialized Agent to route to.


**Reasoning**:
Define the router function to direct the workflow based on the classifier's output.



In [ ]:
def router(state: dict) -> str:
    """Routes the query based on the classifier's output."""
    classification = state.get('classification')
    print(f"--- Router Start ---")
    print(f"Current state: {state}")
    print(f"Received classification: {classification}")

    if classification == "billing":
        next_node = "billing_agent"
    elif classification == "technical":
        next_node = "technical_agent"
    else:
        # Fallback to general agent for any other classification
        next_node = "general_agent"

    print(f"Routing to: {next_node}")
    print(f"--- Router End ---")
    return next_node

## Define final response aggregator

### Subtask:
Create a node to collect the responses from the specialized agents and format them into a single response.


**Reasoning**:
Define the `aggregate_response` function to collect responses from specialized agents and format them into a single string, then update the state and return it.



In [ ]:
def aggregate_response(state: dict) -> dict:
    """Collects responses from specialized agents and formats a single response."""
    print(f"--- Aggregation Node ---")
    print(f"Current state before aggregation: {state}")

    billing_response = state.get('billing_response', "")
    technical_response = state.get('technical_response', "")
    general_response = state.get('general_response', "")

    responses = []
    if billing_response:
        responses.append(f"Billing Response: {billing_response}")
    if technical_response:
        responses.append(f"Technical Response: {technical_response}")
    if general_response:
        responses.append(f"General Response: {general_response}")

    final_response = "\n".join(responses)
    # Update the state with the final response.
    # Ensure this is a dictionary update as expected by StateGraph.
    update_dict = {'final_response': final_response}

    print(f"Aggregated Final Response: {final_response}")
    print(f"Output State Update: {update_dict}")
    print(f"------------------------")
    return update_dict

## Build the langgraph workflow

### Subtask:
Connect the nodes using LangGraph to define the flow of the query through the system.


**Reasoning**:
Connect the nodes using LangGraph to define the flow of the query through the system according to the instructions provided.



In [ ]:
from langgraph.graph import StateGraph

# Instantiate a StateGraph with the defined state
workflow = StateGraph(GraphState)

# Add the previously defined agent nodes and the router node
workflow.add_node("classifier_agent", classifier_agent)
workflow.add_node("billing_agent", billing_agent)
workflow.add_node("technical_agent", technical_agent)
workflow.add_node("general_agent", general_agent)
workflow.add_node("router", router)
workflow.add_node("aggregate_response", aggregate_response) # Add the aggregation node here

# Set the entry point of the graph
workflow.set_entry_point("classifier_agent")

# Add an edge from the classifier_agent to the router
workflow.add_edge("classifier_agent", "router")

# Add conditional edges from the router to the specialized agent nodes
# The router function's string output is used by add_conditional_edges to pick the next node
workflow.add_conditional_edges(
    "router",
    router, # The router function determines the next node by returning its name (string)
    {
        "billing_agent": "billing_agent",
        "technical_agent": "technical_agent",
        "general_agent": "general_agent"
    }
)

# Add edges from each of the specialized agent nodes back to the aggregate_response node
workflow.add_edge("billing_agent", "aggregate_response")
workflow.add_edge("technical_agent", "aggregate_response")
workflow.add_edge("general_agent", "aggregate_response")


# Set the aggregate_response node as the end point of the graph
workflow.set_finish_point("aggregate_response")

# Compile the graph
app = workflow.compile()

## Compile and run the workflow

### Subtask:
Compile the LangGraph workflow and test it with sample customer queries.


**Reasoning**:
Define sample customer queries, iterate through them, and invoke the compiled LangGraph application for each query. Print the input query and the final response.



**Reasoning**:
Review the code from the previous attempt to ensure the router returns a string and that add_conditional_edges is used correctly. Then, check if the agent nodes return dictionaries. Finally, re-compile the graph and test with sample queries while adding print statements for debugging the state updates.



In [ ]:
## FULL CODE IN THIS BLOCK

from langgraph.graph import StateGraph
from typing import TypedDict, Annotated, Union
import operator
from langchain_core.prompts import ChatPromptTemplate

# Define a state for the graph using TypedDict
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        query: initial customer query
        classification: classification of the query
        billing_response: str
        technical_response: str
        general_response: str
        final_response: str
    """
    query: str
    classification: str
    billing_response: str
    technical_response: str
    general_response: str
    final_response: str

# Redefine the classifier_agent to return a dictionary to update the state
def classifier_agent(state: dict) -> dict:
    """Classifies the customer query and updates the state."""
    query = state['query']
    print(f"--- Classifier Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that classifies customer queries into one of the following categories: billing, technical, general. Respond with only the category name."),
        ("human", "Classify the following query: {query}")
    ])
    chain = prompt | openai_llm
    category = chain.invoke({"query": query}).content.strip().lower() # Ensure clean output
    # Basic parsing to extract the category
    if "billing" in category:
        classified_category = "billing"
    elif "technical" in category:
        classified_category = "technical"
    else:
        classified_category = "general"

    print(f"Classified as: {classified_category}")
    print(f"Output State Update: {{'classification': '{classified_category}'}}")
    print(f"--- Classifier Agent End ---")

    # Return a dictionary to update the state
    return {"classification": classified_category}

# Redefine the specialized agents to return dictionaries to update the state
def billing_agent(state: dict) -> dict:
    """Handles billing queries and updates the state."""
    query = state['query']
    print(f"--- Billing Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that responds to billing queries."),
        ("human", "Respond to the following billing query: {query}")
    ])
    chain = prompt | openai_llm
    response = chain.invoke({"query": query}).content
    print(f"Response: {response}")
    print(f"Output State Update: {{'billing_response': '{response}'}}")
    print(f"--- Billing Agent End ---")
    return {"billing_response": response}

def technical_agent(state: dict) -> dict:
    """Handles technical queries and updates the state."""
    query = state['query']
    print(f"--- Technical Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that responds to technical queries."),
        ("human", "Respond to the following technical query: {query}")
    ])
    chain = prompt | openai_llm
    response = chain.invoke({"query": query}).content
    print(f"Response: {response}")
    print(f"Output State Update: {{'technical_response': '{response}'}}")
    print(f"--- Technical Agent End ---")
    return {"technical_response": response}

def general_agent(state: dict) -> dict:
    """Handles general queries and updates the state."""
    query = state['query']
    print(f"--- General Agent Start ---")
    print(f"Input Query: {query}")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that responds to general queries."),
        ("human", "Respond to the following general query: {query}")
    ])
    chain = prompt | openai_llm
    response = chain.invoke({"query": query}).content
    print(f"Response: {response}")
    print(f"Output State Update: {{'general_response': '{response}'}}")
    print(f"--- General Agent End ---")
    return {"general_response": response}

# Redefine the router node
def router(state: dict) -> str:
    """Routes the query based on the classifier's output."""
    classification = state.get('classification')
    print(f"--- Router Start ---")
    print(f"Current state: {state}")
    print(f"Received classification: {classification}")

    if classification == "billing":
        next_node = "billing_agent"
    elif classification == "technical":
        next_node = "technical_agent"
    else:
        # Fallback to general agent for any other classification
        next_node = "general_agent"

    print(f"Routing to: {next_node}")
    print(f"--- Router End ---")
    return next_node

# Redefine the aggregate_response node
def aggregate_response(state: dict) -> dict:
    """Collects responses from specialized agents and formats a single response."""
    print(f"--- Aggregation Node ---")
    print(f"Current state before aggregation: {state}")

    billing_response = state.get('billing_response', "")
    technical_response = state.get('technical_response', "")
    general_response = state.get('general_response', "")

    responses = []
    if billing_response:
        responses.append(f"Billing Response: {billing_response}")
    if technical_response:
        responses.append(f"Technical Response: {technical_response}")
    if general_response:
        responses.append(f"General Response: {general_response}")

    final_response = "\n".join(responses)
    # Update the state with the final response.
    # Ensure this is a dictionary update as expected by StateGraph.
    update_dict = {'final_response': final_response}

    print(f"Aggregated Final Response: {final_response}")
    print(f"Output State Update: {update_dict}")
    print(f"------------------------")
    return update_dict


# Re-compile the graph with the updated nodes
# Instantiate a StateGraph with the defined state
workflow = StateGraph(GraphState)

# Add the updated agent nodes and the router node
workflow.add_node("classifier_agent", classifier_agent)
workflow.add_node("billing_agent", billing_agent)
workflow.add_node("technical_agent", technical_agent)
workflow.add_node("general_agent", general_agent)
workflow.add_node("router", router)
workflow.add_node("aggregate_response", aggregate_response)


# Set the entry point of the graph
workflow.set_entry_point("classifier_agent")
# workflow.add_edge("classifier_agent", "router")

# Instead, use add_conditional_edges from classifier_agent:
workflow.add_conditional_edges(
    "classifier_agent",  # source node
    router,              # routing function
    {
        "billing_agent": "billing_agent",
        "technical_agent": "technical_agent",
        "general_agent": "general_agent"
    }
)

# REMOVE explicit edges from specialized agents to aggregate_response
# workflow.add_edge("billing_agent", "aggregate_response")
# workflow.add_edge("technical_agent", "aggregate_response")
# workflow.add_edge("general_agent", "aggregate_response")

# Set the aggregate_response node as the end point of the graph
# We will now add edges from the specialized agents to the aggregate_response node
workflow.add_edge("billing_agent", "aggregate_response")
workflow.add_edge("technical_agent", "aggregate_response")
workflow.add_edge("general_agent", "aggregate_response")

workflow.set_finish_point("aggregate_response")


# Compile the graph
app = workflow.compile()

# Define sample customer queries
sample_queries = [
    "What is my current bill amount?",  # Billing query
    #"My internet connection is slow, how can I fix it?", # Technical query
    #"What are your operating hours?", # General query
    #"How do I update my payment information?" # Billing query
]

# Iterate through sample queries and test the workflow
for query in sample_queries:
    print(f"=== Running workflow for query: {query} ===")
    # Invoke the compiled LangGraph application
    # The initial state should contain the input query
    result = app.invoke({"query": query})
    # The final response is stored in the 'final_response' key of the final state
    print(f"--- Final Result ---")
    print(f"Input Query: {query}")
    print(f"Final Response: {result.get('final_response', 'No final response generated.')}")
    print(f"===========================================\n")

=== Running workflow for query: What is my current bill amount? ===
--- Classifier Agent Start ---
Input Query: What is my current bill amount?
Classified as: billing
Output State Update: {'classification': 'billing'}
--- Classifier Agent End ---
--- Router Start ---
Current state: {'classification': 'billing', 'query': 'What is my current bill amount?'}
Received classification: billing
Routing to: billing_agent
--- Router End ---
--- Billing Agent Start ---
Input Query: What is my current bill amount?
Response: I don’t have access to your account, so I can’t look up your bill directly. I can help you find it — which account or provider is this for (electric, water, phone, cable, etc.)?

Ways to check your current bill now:
- Online: Sign in to your provider’s website or mobile app → Account/Billing → Current balance or Statements.  
- Email: Check the most recent billing email from the provider (subject often includes “Your bill” or “Statement”).  
- Paper: Look at the most recent mai